In [37]:
!pip install spotipy
!pip install billboard.py
!pip install lyricsgenius
!pip install nltk

In [40]:
import pandas as pd
import numpy as np
import billboard as bb
import lyricsgenius
import nltk
from sklearn.feature_extraction.text import CountVectorizer
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.feature_extraction.text import TfidfVectorizer

In [8]:
genius = lyricsgenius.Genius("JxIIf2vA02Oopvz2JRUbC6iysXRZnNEHN0j84a16uLsB8niafpsNZPR1SvcSxu8r")
genius.verbose = False # Turn off status messages
genius.remove_section_headers = True # Remove section headers (e.g. [Chorus]) from lyrics when searching
genius.skip_non_songs = False # Include hits thought to be non-songs (e.g. track lists)
genius.excluded_terms = ["(Remix)", "(Live)"] # Exclude songs with these words in their title

In [9]:
def create_songs_df(chart_name):
    top100 = bb.ChartData(chart_name)
    songs = top100.entries
    dataframe = {"Rank": [], "Title": [], "Artist": []}
    for song in songs:
        dataframe["Title"].append(song.title)
        dataframe["Artist"].append(song.artist)
        dataframe["Rank"].append(song.rank)
    df = pd.DataFrame(dataframe)
    return df

In [10]:
top100 = create_songs_df("hot-100")
country = create_songs_df("country-songs")
rock = create_songs_df("rock-songs")
pop = create_songs_df("pop-songs")
print(top100)

    Rank              Title                                            Artist
0      1            Circles                                       Post Malone
1      2  Someone You Loved                                     Lewis Capaldi
2      3       Good As Hell                                             Lizzo
3      4           Memories                                          Maroon 5
4      5            Roxanne                                    Arizona Zervas
..   ...                ...                                               ...
95    96    Hot Girl Summer  Megan Thee Stallion, Nicki Minaj & Ty Dolla $ign
96    97       Lonely Child                        YoungBoy Never Broke Again
97    98   Into The Unknown                               Panic! At The Disco
98    99      Show Yourself                   Idina Menzel & Evan Rachel Wood
99   100           Homesick                                        Kane Brown

[100 rows x 3 columns]


In [11]:
def try_load_lyrics(title,artist):
    try:
        return genius.search_song(title, artist).lyrics
    except:
        return "Failed to Find Lyrics"

In [12]:
def populate_chart_with_lyrics(chart):
    chart["lyrics"] = chart.apply(lambda x: try_load_lyrics(x["Title"],x["Artist"]), axis = 1)
    return chart

In [13]:
top100 = populate_chart_with_lyrics(top100)
display(top100)
len(top100[top100['lyrics'] == 'Failed to Find Lyrics'])

,Rank,Title,Artist,lyrics
0,1,Circles,Post Malone,"Oh, oh, oh\nOh, oh, oh\nOh, oh, oh, oh, oh\n..."
1,2,Someone You Loved,Lewis Capaldi,"I'm going under, and this time, I fear there's..."
2,3,Good As Hell,Lizzo,"I do my hair toss, check my nails\nBaby, how y..."
3,4,Memories,Maroon 5,Here's to the ones that we got\nCheers to the ...
4,5,Roxanne,Arizona Zervas,All for the 'Gram\nBitches love the 'Gram\nOh ...
5,6,Lose You To Love Me,Selena Gomez,You promised the world and I fell for it\nI pu...
6,7,Senorita,Shawn Mendes & Camila Cabello,I love it when you call me señorita\nI wish I ...
7,8,"10,000 Hours",Dan + Shay & Justin Bieber,"Do you love the rain, does it make you dance\n..."
8,9,Truth Hurts,Lizzo,Why're men great 'til they gotta be great?\nWo...
9,10,No Guidance,Chris Brown Featuring Drake,My name is Devvon Terrell\nThis is a raw sessi...


1

In [15]:
client_credentials_manager = SpotifyClientCredentials(client_id = "ee9a1f8e2f634959967d52ed9fad54de", client_secret = "934a6bbc90e74664b502638f6f5fc92b")
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [17]:
def get_spotify_popularity(title):
    title = title.replace("$","s")
    answer = sp.search(title, type="track", limit = 1)
    track = sp.track(answer["tracks"]["items"][0]["id"])
    return track["popularity"]

In [18]:
def populate_chart_with_spotify_popularity(chart):
    chart["spotify_popularity"] = chart.apply(lambda x: get_spotify_popularity(x["Title"]), axis = 1)
    return chart

In [19]:
top100 = populate_chart_with_spotify_popularity(top100)
display(top100)

,Rank,Title,Artist,lyrics,spotify_popularity
0,1,Circles,Post Malone,"Oh, oh, oh\nOh, oh, oh\nOh, oh, oh, oh, oh\n...",99
1,2,Someone You Loved,Lewis Capaldi,"I'm going under, and this time, I fear there's...",96
2,3,Good As Hell,Lizzo,"I do my hair toss, check my nails\nBaby, how y...",86
3,4,Memories,Maroon 5,Here's to the ones that we got\nCheers to the ...,100
4,5,Roxanne,Arizona Zervas,All for the 'Gram\nBitches love the 'Gram\nOh ...,96
5,6,Lose You To Love Me,Selena Gomez,You promised the world and I fell for it\nI pu...,98
6,7,Senorita,Shawn Mendes & Camila Cabello,I love it when you call me señorita\nI wish I ...,96
7,8,"10,000 Hours",Dan + Shay & Justin Bieber,"Do you love the rain, does it make you dance\n...",95
8,9,Truth Hurts,Lizzo,Why're men great 'til they gotta be great?\nWo...,91
9,10,No Guidance,Chris Brown Featuring Drake,My name is Devvon Terrell\nThis is a raw sessi...,87


In [35]:
def create_term_doc_matrix(chart):
    lyrics = chart["lyrics"].to_list()
    vec = CountVectorizer()
    X = vec.fit_transform(lyrics)
    termdocmatrix = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index=top100.Title)
    return termdocmatrix

top100 = top100[top100.lyrics != "Failed to Find Lyrics"]
create_term_doc_matrix(top100)

,000,02,10,100,101,10k,11,12,12am,13,...,کردن,کردیم,کسی,کلی,کمگ,کنار,کنیم,که,گرفتن,یه
Title,,,,,,,,,,,,,,,,,,,,,
Circles,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Someone You Loved,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Good As Hell,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Memories,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Roxanne,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Lose You To Love Me,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Senorita,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
"10,000 Hours",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Truth Hurts,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [41]:
def create_tfidf(chart):
    lyrics = chart["lyrics"].to_list()
    vectorizer = TfidfVectorizer()
    doc_vec = vectorizer.fit_transform(lyrics)
    tfidf = pd.DataFrame(doc_vec.toarray().transpose(), index=vectorizer.get_feature_names())
    tfidf.columns = chart["Title"]
    return tfidf

tfidf = create_tfidf(top100)
tfidf

Title,Circles,Someone You Loved,Good As Hell,Memories,Roxanne,Lose You To Love Me,Senorita,"10,000 Hours",Truth Hurts,No Guidance,...,Make No Sense,Good Vibes,La Cancion,Easy,Cuban Links,Hot Girl Summer,Lonely Child,Into The Unknown,Show Yourself,Homesick
000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.008303,0.000000,0.00000,0.0,0.0,0.0
02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.008303,0.000000,0.00000,0.0,0.0,0.0
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.222900,0.043349,0.00000,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027678,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.045770,0.00000,0.0,0.0,0.0
101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.045265,0.000000,0.000000,0.00000,0.0,0.0,0.0
10k,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.007618,0.000000,0.00000,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.008303,0.000000,0.00000,0.0,0.0,0.0
12am,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.008303,0.000000,0.00000,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0


In [ ]:
def cosine_similarity(tfidf):
    similarity = 0
    count = 0
    for index in range(tfidf.shape[1]):
        col = tfidf.iloc[: , index]
        for ind2 in range(index + 1, tfidf.shape[1]):
            col2 = tfidf.iloc[: , ind2]
            similarity = similarity + np.dot(col.values,col2.values)/(np.linalg.norm(col.values)*np.linalg.norm(col2.values))
            count = count + 1
            
            